In [2]:
import mlflow
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import os

In [3]:
mlflow.set_tracking_uri('http://localhost:5000')

os.environ['AWS_ACCESS_KEY_ID'] = 'minio_user'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio_password'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
endpoint = 'http://localhost:9000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = endpoint

In [4]:
model_name = "my-registered-model"

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names) 

iris_train = iris_df.iloc[:-1]
iris_test = iris_df.iloc[-1:]

experiment_id = mlflow.set_experiment('model-demo')
with mlflow.start_run():
    clf = RandomForestClassifier(max_depth=7, random_state=0)
    clf.fit(iris_train, iris.target[:-1])
    signature = infer_signature(iris_train, clf.predict(iris_train))
    mlflow.sklearn.log_model(clf, "iris_rf", signature=signature, registered_model_name=model_name)

Registered model 'my-registered-model' already exists. Creating a new version of this model...
2025/02/20 22:55:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: my-registered-model, version 2


🏃 View run indecisive-ant-386 at: http://localhost:5000/#/experiments/1/runs/bc3837f82a78459cb19ddc12234e487f
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '2' of model 'my-registered-model'.


In [5]:
signature

inputs: 
  ['sepal length (cm)': double (required), 'sepal width (cm)': double (required), 'petal length (cm)': double (required), 'petal width (cm)': double (required)]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [6]:
import mlflow.pyfunc

model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

result = model.predict(iris_test)

print(result)

[2]


In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

stage = 'Staging'

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage
)

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

result = model.predict(iris_test)

print(result)

[2]


/tmp/ipykernel_122687/2333998311.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


In [9]:
from pprint import pprint

for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1739708934471,
    'description': '',
    'last_updated_timestamp': 1739708942072,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1739708934539, current_stage='Staging', description='', last_updated_timestamp=1739708942072, name='my-registered-model', run_id='82b5ecff69894b6bab9517e2410c42b6', run_link='', source='s3://bucket/1/82b5ecff69894b6bab9517e2410c42b6/artifacts/iris_rf', status='READY', status_message=None, tags={}, user_id='', version='1'>],
    'name': 'my-registered-model',
    'tags': {}}


In [ ]:
client.delete_registered_model(name=model_name)

In [ ]:
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)